In [102]:
import os
os.chdir('/Users/ltran/Documents/TrueData29/CPC_ML_tutorial/')

import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import random
import joypy
from matplotlib import cm

from scipy.stats import fisher_exact
from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d
from sklearn.model_selection import train_test_split

# Set working directory
root_dir = '/Users/ltran/Documents/TrueData0104/CPC_ML_tutorial/'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

# Load TCA

In [192]:
data_dir = '/Users/ltran/Documents/Data/'


## Compute BMI

In [193]:
pheno = pd.read_csv(os.path.join(data_dir, 'TCA_vol/MRI_RDB_TCA_20220629_anonym.tsv'), sep = '\t')

In [194]:
# Problème pour la taille (m -> cm)
pheno['size_at_scan'] = pheno['size_at_scan'].replace(0.92, 92)

In [195]:
# Rajouter suffixe à participant_id
pheno['participant_id'] = pheno['subject_id'].astype(str)+'_tca'

In [196]:
S = []
for i in pheno.subject_id:
    S.append('sub-'+str(i).zfill(4))
    
pheno['Subject'] = S

## Remove failed

In [197]:
list_fail = [8, 12, 18, 152, 26]

In [198]:
pheno = pheno[~(pheno.subject_id.isin(list_fail))]

# Merge Volume, Area, CT

In [199]:
# Load ISO volume
vol_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/volumes-fs-iso.txt'), sep = '\t')

# Load TFE volume
vol_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/volumes-fs-tfe.txt'), sep = '\t')

In [200]:
# Merge volume ISO + TFE 

vol_tca = pd.concat([vol_tfe, vol_iso])

In [201]:
lh_area_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.lh.area.csv'), sep = '\t') 
rh_area_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.rh.area.csv'), sep = '\t') 


lh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.lh.area.csv'), sep = '\t') 
rh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.rh.area.csv'), sep = '\t') 

area_iso = lh_area_iso.merge(rh_area_iso.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

area_tfe = lh_area_tfe.merge(rh_area_tfe.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

In [202]:
area = pd.concat([area_iso, area_tfe])
area = area.rename(columns= {'lh.aparc.area' : 'Subject'})

In [217]:
lh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.lh.thickness.csv'), sep = '\t') 
rh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.rh.thickness.csv'), sep = '\t') 

ct_iso = lh_ct_iso.merge(rh_ct_iso.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), 
               on = 'lh.aparc.thickness')


lh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.lh.thickness.csv'), sep = '\t') 
rh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.rh.thickness.csv'), sep = '\t') 

ct_tfe = lh_ct_tfe.merge(rh_ct_tfe.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), 
               on = 'lh.aparc.thickness')


ct = pd.concat([ct_iso, ct_tfe])
ct = ct.rename(columns= {'lh.aparc.thickness' : 'Subject'})

In [218]:
df_TCA = area.merge(vol_tca, on = 'Subject').merge(ct, on='Subject')

In [219]:
l_pi = []
for i in df_TCA['Subject']:
    l_pi.append(int(i.split('-')[1].split('_')[0]))

df_TCA['participant_id'] = l_pi

In [220]:
df_TCA['participant_id'] = df_TCA['participant_id'].astype(str)+'_tca'

In [221]:
df_TCA = df_TCA.merge(pheno, on = 'participant_id')

In [222]:
df_TCA['sex'] = df_TCA['Sex'].replace({'Female' : 2, 'Male' : 1})

In [223]:
# df_TCA = df_TCA.drop_duplicates(subset='participant_id')
df_TCA = df_TCA.dropna(subset='Sex')

In [224]:
df_TCA.columns = df_TCA.columns.str.replace('-', '_')

In [225]:
df_TCA['ASD'] = 'TCA'
df_TCA.loc[df_TCA[df_TCA.TSA == 1].index, 'ASD'] = 'TCA_Autism'

In [226]:
# df_TCA = df_TCA[~(df_TCA['magnetic_field_strength'] == '3')]

In [227]:
df_TCA = df_TCA.drop_duplicates(subset='participant_id', keep = 'first')

In [228]:
df_TCA['size_at_scan'] = df_TCA['size_at_scan'].replace(0.92, 92)

In [229]:
df_TCA['Month'] = df_TCA['age_at_scan']*12

In [230]:
df_TCA['scanner'] = df_TCA['manufacturer_model_name']

In [231]:
df_TCA = df_TCA.rename(columns= {'3rd_Ventricle' : 'third_Ventricle',
                       '4th_Ventricle' : 'fourth_Ventricle',
                        '5th_Ventricle' : 'fifth_Ventricle'})

### Export dataframe

In [232]:
data_dir

'/Users/ltran/Documents/Data/'

In [233]:
df_TCA.to_csv(os.path.join(data_dir, 'TCA_vol/Outputs/df_TCA.csv'), index = False)

________________

# Load TSA

Load infos 

In [234]:
def is_duplicated(df, col):
    return df[df.duplicated(subset = col, keep = False)]
    

In [235]:
infos_TSA = pd.read_csv(os.path.join(data_dir, 'TSA_vol/MRI_RDB_20220928_anonym.tsv'), sep = '\t')

In [236]:
infos_TSA['Subject'] = 'sub-'+infos_TSA['subject_id'].astype(str).str.zfill(4)+'_ses-0'+infos_TSA['session_id'].astype(str
                                                                    )

In [237]:
# Enleve les NA dans sex, ASD, age
infos_TSA = infos_TSA.dropna(subset = ['Sex', 'age_at_scan', 'ASD'])
infos_TSA = infos_TSA[~(infos_TSA.ASD == '?')]

In [238]:
infos_TSA['sex'] = infos_TSA.Sex.replace({'Male' : 1, 'Female' : 2})

In [239]:
infos_TSA.loc[(infos_TSA.Group == 'Relative') & (infos_TSA.ASD == 'No'), 'ASD'] = 'Relative'

Load QC

In [240]:
QC = pd.read_csv(os.path.join(data_dir, 'QC_trio.csv'), sep = ',')

In [241]:
bids_f = QC[(QC.QC == 'F') | (QC.QC == 'g')][['BIDS', 'session']]

In [242]:
p = []

for i in bids_f.BIDS:
    if (type(i) == str):
        p.append(int(i.split('-')[1]))
    else:
        p.append(i)
bids_f['subject_id'] = p



In [243]:
bids_f = bids_f.rename(columns={'session' : 'session_id'})

In [244]:
infos_TSA = infos_TSA[~(infos_TSA.subject_id.isin(bids_f.subject_id))]

In [245]:
# Enlever les duplicata sur les sujets

infos_TSA = infos_TSA.drop_duplicates(subset='subject_id', keep  = 'first')

Volumes

In [246]:
# Load ISO volume
vol_tsa_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/volumes-fs-iso.txt'), sep = '\t')

# Load TFE volume
vol_tsa_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/volumes-fs-tfe.txt'), sep = '\t')

In [247]:
vol_tsa = pd.concat([vol_tsa_iso, vol_tsa_tfe])

In [248]:
vol_tsa = vol_tsa.drop_duplicates(subset= 'Subject')

Surface area

In [249]:
rh_area_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.rh.area.csv'), sep = '\t')
lh_area_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.lh.area.csv'), sep = '\t')
rh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.rh.area.csv'), sep = '\t')
lh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.lh.area.csv'), sep = '\t')

In [250]:
area_iso = lh_area_iso.merge(rh_area_iso.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

In [251]:
area_tfe = lh_area_tfe.merge(rh_area_tfe.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

In [252]:
area_iso = area_iso.rename(columns={'lh.aparc.area':'Subject'})
area_tfe = area_tfe.rename(columns={'lh.aparc.area':'Subject'})

In [253]:
area = pd.concat([area_iso,area_tfe])

In [254]:
area = area.drop_duplicates(subset= 'Subject')

Cortical thickness

In [255]:
rh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.rh.thickness.csv'), sep = '\t')
lh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.lh.thickness.csv'), sep = '\t')
rh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.rh.thickness.csv'), sep = '\t')
lh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.lh.thickness.csv'), sep = '\t')

In [256]:
ct_iso = lh_ct_iso.merge(rh_ct_iso.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), on ='lh.aparc.thickness')

In [257]:
ct_tfe = lh_ct_tfe.merge(rh_ct_tfe.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), on ='lh.aparc.thickness')

In [258]:
ct_iso = ct_iso.rename(columns={'lh.aparc.thickness':'Subject'})
ct_tfe = ct_tfe.rename(columns={'lh.aparc.thickness':'Subject'})

In [259]:
ct_tsa = pd.concat([ct_iso, ct_tfe])

In [260]:
ct_tsa = ct_tsa.drop_duplicates(subset= 'Subject')

### Merge volume / area / ct

In [268]:
df_tsa = area.merge(ct_tsa, on = 'Subject').merge(vol_tsa, on = 'Subject')

In [269]:
participants = []
for i in df_tsa['Subject']:
    participants.append(int(i.split('-')[1].split('_')[0]))
    
df_tsa['participant_id'] = participants
df_tsa['participant_id'] = df_tsa['participant_id'].astype(str)+'_tsa'

In [270]:
infos_TSA['participant_id'] = infos_TSA['subject_id'].astype(str)+'_tsa'

In [271]:
df_tsa = df_tsa.merge(infos_TSA,
                                   on  = 'participant_id')

In [272]:
df_tsa = df_tsa.drop_duplicates(subset = 'participant_id', keep = 'first')

In [273]:
df_tsa.columns = df_tsa.columns.str.replace('-', '_')

In [274]:
df_tsa['scanner'] = df_tsa['machine'].str.upper()

In [275]:
df_tsa = df_tsa.rename(columns= {'3rd_Ventricle' : 'third_Ventricle',
                       '4th_Ventricle' : 'fourth_Ventricle',
                        '5th_Ventricle' : 'fifth_Ventricle'})

## Export TSA

In [276]:
df_tsa.to_csv(os.path.join(data_dir, 'TSA_vol/Outputs/df_TSA.csv'), index = False)

# TCA + TSA

In [277]:
df_tsa_tca = pd.concat([df_tsa, df_TCA])


In [176]:
df_tsa_tca.groupby('ASD').size().to_frame('Count').reset_index().rename(columns = {'ASD': 'Diag'})

,Diag,Count
0,No,190
1,Relative,205
2,TCA,105
3,TCA_Autism,4
4,Yes,608


## Clean datasets for linear regression

In [177]:
df_tsa_tca = df_tsa_tca[~((df_tsa_tca['magnetic_field_strength'] == '3')|(df_tsa_tca['machine'] == 'Ingenia_3T'))]

In [178]:
df_tsa_tca = df_tsa_tca.rename(columns={'3rd_Ventricle':'third_Ventricle',
                           '4th_Ventricle' : 'fourth_Ventricle',
                           '5th_Ventricle' : 'fifth_Ventricle'
    
})

In [179]:
df_tsa_tca.groupby('ASD').size().to_frame('Count').reset_index().rename(columns = {'ASD': 'Diag'})

,Diag,Count
0,No,190
1,Relative,205
2,TCA,89
3,TCA_Autism,3
4,Yes,552


In [180]:
thick = [col for col in df_tsa_tca.columns if '_thickness' in col]
thick.remove('lh_MeanThickness_thickness')
thick.remove('rh_MeanThickness_thickness')
len(thick)

68

In [181]:
df_tsa_tca['meanCT'] = df_tsa_tca[['lh_MeanThickness_thickness', 'rh_MeanThickness_thickness']].mean(axis=1)

In [182]:
area  = [col for col in df_tsa_tca.columns if '_area' in col]
area.remove('rh_WhiteSurfArea_area')
area.remove('lh_WhiteSurfArea_area')
area.remove('Left_Accumbens_area'),
area.remove('Right_Accumbens_area')

In [183]:
df_tsa_tca['TotalSA'] = df_tsa_tca[area].sum(axis =1)

## Export TSA + TCA

In [185]:
df_tsa_tca.to_csv(os.path.join(data_dir, 'Outputs/df_tsa_tca.csv'))

In [96]:
for i in thick:
    print(i+',')

lh_bankssts_thickness,
lh_caudalanteriorcingulate_thickness,
lh_caudalmiddlefrontal_thickness,
lh_cuneus_thickness,
lh_entorhinal_thickness,
lh_fusiform_thickness,
lh_inferiorparietal_thickness,
lh_inferiortemporal_thickness,
lh_isthmuscingulate_thickness,
lh_lateraloccipital_thickness,
lh_lateralorbitofrontal_thickness,
lh_lingual_thickness,
lh_medialorbitofrontal_thickness,
lh_middletemporal_thickness,
lh_parahippocampal_thickness,
lh_paracentral_thickness,
lh_parsopercularis_thickness,
lh_parsorbitalis_thickness,
lh_parstriangularis_thickness,
lh_pericalcarine_thickness,
lh_postcentral_thickness,
lh_posteriorcingulate_thickness,
lh_precentral_thickness,
lh_precuneus_thickness,
lh_rostralanteriorcingulate_thickness,
lh_rostralmiddlefrontal_thickness,
lh_superiorfrontal_thickness,
lh_superiorparietal_thickness,
lh_superiortemporal_thickness,
lh_supramarginal_thickness,
lh_frontalpole_thickness,
lh_temporalpole_thickness,
lh_transversetemporal_thickness,
lh_insula_thickness,
rh_banksst

In [97]:
thick

['lh_bankssts_thickness',
 'lh_caudalanteriorcingulate_thickness',
 'lh_caudalmiddlefrontal_thickness',
 'lh_cuneus_thickness',
 'lh_entorhinal_thickness',
 'lh_fusiform_thickness',
 'lh_inferiorparietal_thickness',
 'lh_inferiortemporal_thickness',
 'lh_isthmuscingulate_thickness',
 'lh_lateraloccipital_thickness',
 'lh_lateralorbitofrontal_thickness',
 'lh_lingual_thickness',
 'lh_medialorbitofrontal_thickness',
 'lh_middletemporal_thickness',
 'lh_parahippocampal_thickness',
 'lh_paracentral_thickness',
 'lh_parsopercularis_thickness',
 'lh_parsorbitalis_thickness',
 'lh_parstriangularis_thickness',
 'lh_pericalcarine_thickness',
 'lh_postcentral_thickness',
 'lh_posteriorcingulate_thickness',
 'lh_precentral_thickness',
 'lh_precuneus_thickness',
 'lh_rostralanteriorcingulate_thickness',
 'lh_rostralmiddlefrontal_thickness',
 'lh_superiorfrontal_thickness',
 'lh_superiorparietal_thickness',
 'lh_superiortemporal_thickness',
 'lh_supramarginal_thickness',
 'lh_frontalpole_thickness'

In [98]:
vol = ['Left_Lateral_Ventricle', 'Left_Inf_Lat_Vent',
       'Left_Cerebellum_White_Matter', 'Left_Cerebellum_Cortex',
       'Left_Thalamus', 'Left_Caudate', 'Left_Putamen', 'Left_Pallidum',
       'third_Ventricle', 'fourth_Ventricle', 'Brain_Stem', 'Left_Hippocampus',
       'Left_Amygdala', 'CSF', 'Left_Accumbens_area', 'Left_VentralDC',
       'Left_vessel', 'Left_choroid_plexus', 'Right_Lateral_Ventricle',
       'Right_Inf_Lat_Vent', 'Right_Cerebellum_White_Matter',
       'Right_Cerebellum_Cortex', 'Right_Thalamus', 'Right_Caudate',
       'Right_Putamen', 'Right_Pallidum', 'Right_Hippocampus',
       'Right_Amygdala', 'Right_Accumbens_area', 'Right_VentralDC',
       'Right_vessel', 'Right_choroid_plexus', 'fifth_Ventricle']

In [99]:
df_tsa_tca[vol]

,Left_Lateral_Ventricle,Left_Inf_Lat_Vent,Left_Cerebellum_White_Matter,Left_Cerebellum_Cortex,Left_Thalamus,Left_Caudate,Left_Putamen,Left_Pallidum,third_Ventricle,fourth_Ventricle,...,Right_Caudate,Right_Putamen,Right_Pallidum,Right_Hippocampus,Right_Amygdala,Right_Accumbens_area,Right_VentralDC,Right_vessel,Right_choroid_plexus,fifth_Ventricle
0,5600.0,261.3,13449.9,51571.3,8303.4,2889.7,4512.3,2171.8,628.3,1708.8,...,3000.6,4983.5,1926.1,4106.4,1899.0,534.8,3494.1,14.5,449.3,0.0
1,3671.9,143.0,12312.3,56378.2,7775.8,3912.7,5582.6,2489.1,507.4,1125.9,...,4533.7,5509.2,2528.8,4099.9,1759.1,557.7,4384.3,171.6,388.8,3.6
2,3924.3,176.8,13970.7,66288.8,8257.7,4309.3,5652.9,2163.5,803.3,1803.6,...,4513.2,5696.5,2144.2,4450.4,1491.9,496.7,3567.2,20.9,367.1,0.0
4,19052.9,318.6,14572.6,63181.6,8647.0,3971.6,6025.6,2202.8,922.3,3682.3,...,4023.1,5887.9,2374.2,5078.5,1996.3,603.4,4140.0,10.0,723.2,0.0
5,2686.4,0.0,20251.6,47612.1,7666.1,1397.2,6193.3,2753.1,602.3,600.1,...,2399.2,3426.8,2402.9,2269.2,1004.5,224.4,3455.3,23.0,673.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,3515.1,327.6,11938.0,56824.1,7121.7,4005.6,5425.3,2111.2,888.6,1870.1,...,4167.2,5634.2,1985.5,3891.4,1491.2,540.2,3674.3,32.2,624.3,0.0
95,7597.2,373.2,13533.3,54702.4,8500.2,3808.9,6660.2,2263.9,609.3,1438.0,...,3963.7,6723.5,2233.0,4015.0,1394.5,737.2,4415.4,46.8,738.5,0.0
98,10699.5,596.0,13603.5,62036.7,8018.7,4094.8,5563.5,1996.1,2014.0,1839.7,...,4262.8,5545.9,1886.7,3928.8,1664.6,634.4,4313.3,17.1,512.0,0.0
99,6714.4,441.1,15091.2,64797.6,7398.6,3833.8,6148.5,1977.8,1106.3,3020.5,...,3970.0,6273.7,2053.4,4159.6,1808.4,596.5,3944.8,23.7,606.5,0.0


In [100]:
for i in vol:
    print(i+',')

Left_Lateral_Ventricle,
Left_Inf_Lat_Vent,
Left_Cerebellum_White_Matter,
Left_Cerebellum_Cortex,
Left_Thalamus,
Left_Caudate,
Left_Putamen,
Left_Pallidum,
third_Ventricle,
fourth_Ventricle,
Brain_Stem,
Left_Hippocampus,
Left_Amygdala,
CSF,
Left_Accumbens_area,
Left_VentralDC,
Left_vessel,
Left_choroid_plexus,
Right_Lateral_Ventricle,
Right_Inf_Lat_Vent,
Right_Cerebellum_White_Matter,
Right_Cerebellum_Cortex,
Right_Thalamus,
Right_Caudate,
Right_Putamen,
Right_Pallidum,
Right_Hippocampus,
Right_Amygdala,
Right_Accumbens_area,
Right_VentralDC,
Right_vessel,
Right_choroid_plexus,
fifth_Ventricle,


In [101]:
vol_tsa.columns.str.replace('-','_')

Index(['Subject', 'BrainSeg', 'BrainSegNotVent', 'VentricleChoroidVol',
       'lhCortex', 'rhCortex', 'Cortex', 'lhCerebralWhiteMatter',
       'rhCerebralWhiteMatter', 'CerebralWhiteMatter', 'SubCortGray',
       'TotalGray', 'SupraTentorial', 'SupraTentorialNotVent', 'Mask',
       'BrainSegVol_to_eTIV', 'MaskVol_to_eTIV', 'lhSurfaceHoles',
       'rhSurfaceHoles', 'SurfaceHoles', 'EstimatedTotalIntraCranialVol',
       'Left_Lateral_Ventricle', 'Left_Inf_Lat_Vent',
       'Left_Cerebellum_White_Matter', 'Left_Cerebellum_Cortex',
       'Left_Thalamus', 'Left_Caudate', 'Left_Putamen', 'Left_Pallidum',
       '3rd_Ventricle', '4th_Ventricle', 'Brain_Stem', 'Left_Hippocampus',
       'Left_Amygdala', 'CSF', 'Left_Accumbens_area', 'Left_VentralDC',
       'Left_vessel', 'Left_choroid_plexus', 'Right_Lateral_Ventricle',
       'Right_Inf_Lat_Vent', 'Right_Cerebellum_White_Matter',
       'Right_Cerebellum_Cortex', 'Right_Thalamus', 'Right_Caudate',
       'Right_Putamen', 'Right_Pallidu